In [1]:
!pip install pennylane-qiskit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 40.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.7/229.7 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.9/245.9 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 32.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 48.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.4/39.4 MB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━

Handling Noise Models

In [5]:
from qiskit.providers.fake_provider import FakeWashingtonV2
import qiskit_aer.noise as noise

noisemodel=noise.NoiseModel.from_backend(FakeWashingtonV2())
noisemodel=noisemodel.to_dict()

# print(noisemodel)

# Specific noise
print(noisemodel['errors'][7])

{'type': 'qerror', 'id': '088e6d304daa4eb9aeafd7a563010660', 'operations': ['id'], 'instructions': [[{'name': 'id', 'qubits': [0]}, {'name': 'id', 'qubits': [0]}], [{'name': 'id', 'qubits': [0]}, {'name': 'z', 'qubits': [0]}], [{'name': 'id', 'qubits': [0]}, {'name': 'reset', 'qubits': [0]}], [{'name': 'x', 'qubits': [0]}, {'name': 'id', 'qubits': [0]}], [{'name': 'x', 'qubits': [0]}, {'name': 'z', 'qubits': [0]}], [{'name': 'x', 'qubits': [0]}, {'name': 'reset', 'qubits': [0]}], [{'name': 'y', 'qubits': [0]}, {'name': 'id', 'qubits': [0]}], [{'name': 'y', 'qubits': [0]}, {'name': 'z', 'qubits': [0]}], [{'name': 'y', 'qubits': [0]}, {'name': 'reset', 'qubits': [0]}], [{'name': 'z', 'qubits': [0]}, {'name': 'id', 'qubits': [0]}], [{'name': 'z', 'qubits': [0]}, {'name': 'z', 'qubits': [0]}], [{'name': 'z', 'qubits': [0]}, {'name': 'reset', 'qubits': [0]}]], 'probabilities': [0.9995074349213914, 0.00011936718841310697, 0.0003617610560494359, 3.8104438330945823e-06, 4.5506611664007755e-10,

In [1]:
#other than pennylane
"""
np.save("address",obj)
np.load("address")
plt.savefig("address")
"""

In [ ]:
class ModelProcessing:
    def __init__ (self,model,xtrain,ytrain,xtest,ytest,trained=False,intialweights1=None,initialweights2=None):
        # Take all the nessecary objects and define them within a processor object
        self.model=model
        self.xtrain=xtrain
        self.xtest=xtest
        self.ytrain=ytrain
        self.ytest=ytest

        self.trained=trained
        self.firstweight=intialweights1
        self.secondweight=initialweights2

        # If model already trained and initial weights are known then we can skip building weights
        if not self.trained:
          # This step is to just build some weights for the model
          self.model.fit(xtrain[0:1],ytrain[0:1],epochs=1,batch_size=1)
        # Here, we extract weights of the model, the main purpose is to know the formate of the weights for the given circuit.
        self.weights=self.model.get_weights()
        self.landscape_mesh=None

    # Generate Random weights of shape `weights` and in between -1 to 1
    def GenerateRandomWeights(self):
        res=[]
        for x in self.weights:
            res.append(np.random.uniform(-1,1,size=x.shape))
        return res

    # Genrate Zero weights (all values zero) of shape `weights`
    def Zeros(self):
        res=[]
        for x in self.weights:
            res.append(np.zeros(size=x.shape))
        return res

    # Calculate distance between two weight vectors in weight hyperplane
    def DistanceWeights(self,w1,w2):
        res=0
        for i,x in enumerate(w1):
            t=x-w2[i]
            res+=np.sum(np.square(t))
        return np.sqrt(res)

    # Flatten the weights into single list, helpful while handling weights
    def FlattenWeights(self,w):
        res=np.array([])
        for x in w:
            t=x.copy()
            t=t.flatten()
            res=np.concatenate([res,t])
        return res

    # Convert weights in form of Flattenweights to the form of `weights`
    def DeflattenWeights(self,w):
        lp=0
        res=[]
        for x in self.weights:
            s=x.shape
            ni=1
            for y in s: ni*=y
            t=w[lp:lp+ni]
            t=np.reshape(t,s)
            lp+=ni
            res.append(t)
        return res

    # Find a perpendicular direction to v weights in the plane formed by two vector v and u in weight hyperplane
    def PerpendicularDirection(v,u):
        t=v/np.dot(v,v)*np.dot(v,u)
        return u-t

    # Find mid point of two weights
    def avg(self,u,v):

      return self.DeflattenWeights(
            ( self.FlattenWeights(u) + self.FlattenWeights(v) )/2
        )

    # Train the model, also generate initial weights
    def train(self,epoch=40,batchsize=1,shuffle=True,zero_centred=False):
        # Initial weights calcualtion
        self.trained=True
        self.firstweight=self.GenerateRandomWeights()
        self.secondweight= self.GenerateRandomWeights() if not zero_centred else self.Zeros()
        self.model.set_weights(self.avg(self.firstweight,self.secondweight))

        # Train the Model
        hybrid=self.model.fit(
                    self.xtrain,
                    self.ytrain,
                    epochs = epoch,
                    batch_size = batchsize,
                    validation_data=(self.xtest,self.ytest),
                    verbose=0,
                    shuffle = shuffle,workers=cpu_count(), use_multiprocessing=True
        )

    # Generate landscape with first and second weightts as two corners and other two on opposite side of final weights
    def get_landscape(self,dim=10):
        # Evaluate the 4 corner points for the landscape
        centre=self.model.get_weights()
        A=self.FlattenWeights(self.firstweight)
        O=self.FlattenWeights(centre)
        D=self.FlattenWeights(self.secondweight)

        # Evaluate opposite corners of the landscape
        C=A+2*(O-A)
        B=D+2*(O-D)
        landscape=np.zeros((dim,dim))

        # Evaluating loss for each point on the dim*dim surface
        for i in range(dim):
            for j in range(dim):
                a=i/(dim-1)
                b=j/(dim-1)
                temp= (a*b * A) + ((1-a)*b * B) + (a*(1-b) * D) + ((1-a)*(1-b) * C)

                temp=self.DeflattenWeights(temp)
                self.model.set_weights(temp)
                landscape[dim-1-j, dim-1-i] = self.model.evaluate(self.xtest,self.ytest,batch_size=1,verbose=1)[0]


                gc.collect()
            self.model.set_weights(centre)

        return landscape

    # Draw landscape with matplotlib
    def draw(self):
        plt.imshow(self.landscape_mesh, cmap='jet', extent=[-1, 1, -1, 1])
        plt.xlabel('Direction 1')
        plt.ylabel('Direction 2')
        plt.colorbar(label='Loss')
        plt.title('Loss Landscape')

        t=self.landscape_mesh.tolist()
        print('[',end=" ")
        for x in t:
          print(x,',')
        print(']')

    # Calculate loss Surface
    def compute_loss_landscape(self,epoch=40,
                               batchsize=1,dim=10,shuffle=True,zero_centred=False):

        if not self.trained:
            self.train(epoch,batchsize,shuffle,zero_centred)
        print(self.firstweight,self.secondweight,self.model.get_weights())
        if not self.landscape_mesh:
            self.landscape_mesh=self.get_landscape(dim)
        self.draw()
